[demo](https://cloud.baidu.com/qianfandev/topic/267330)

In [13]:
import os

os.environ["QIANFAN_AK"] = "LU4ZTVJKFROUZMJzA0biqqmC"
os.environ["QIANFAN_SK"] = "YHJ33uMEoH4HNTQGdthdGk07fyMW1DbR"

# 此处为 Langsmith 相关功能开关。当且仅当你知道这是做什么用时，可删除注释并设置变量以使用 Langsmith 相关功能
# os.environ['LANGCHAIN_TRACING_V2'] = "true"
# os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
# os.environ['LANGCHAIN_API_KEY'] = "your_langchian_api_key"
# os.environ['LANGCHAIN_PROJECT'] = "your_project_name"

is_chinese = True

if is_chinese:
    WEB_URL = "https://zhuanlan.zhihu.com/p/85289282"
    CUSTOM_PROMPT_TEMPLATE = """
        使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
        
        {context}
        
        请问：{question}
    """
    QUESTION1 = "今天北京的天气怎么样,同时回答今天日期"
    QUESTION2 = "今天北京市朝阳区的天气怎么样"
else:
    WEB_URL = "https://lilianweng.github.io/posts/2023-06-23-agent/"
    CUSTOM_PROMPT_TEMPLATE = """
        Use the following pieces of context to answer the question at the end. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. 
        Use three sentences maximum and keep the answer as concise as possible. 
        Always say "thanks for asking!" at the end of the answer. 
        {context}
        Question: {question}
        Helpful Answer:
    """
    QUESTION1 = "how many firut there?"
    QUESTION2 = "what is the name of the first fruit?"

In [14]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(file_path="/root/LangChainStudy/documentstore/t_china_city_weather.csv",
                   
        csv_args={
        "delimiter": ",",
        "fieldnames": [
            "city"
            ,"adcode"	
            ,"province"	
            ,"reporttime"	
            ,"date"	
            ,"dayweather"
            ,"week","nightweather","daytemp","nighttemp"
            ,"daywind"
            ,"nightwind"
            ,"daypower"
            ,"nightpower"
            ,"daytemp_float"
            ,"nighttemp_float"
            ],
    },
                   )
data = loader.load()
print(data)

[Document(page_content='city: city\nadcode: adcode\nprovince: province\nreporttime: reporttime\ndate: date\ndayweather: dayweather\nweek: week\nnightweather: nightweather\ndaytemp: daytemp\nnighttemp: nighttemp\ndaywind: daywind\nnightwind: nightwind\ndaypower: daypower\nnightpower: nightpower\ndaytemp_float: daytemp_float\nnighttemp_float: nighttemp_float', metadata={'source': '/root/LangChainStudy/documentstore/t_china_city_weather.csv', 'row': 0}), Document(page_content='city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-04-29 13:33:21\ndate: 2024-04-29\ndayweather: 小雨\nweek: 1\nnightweather: 多云\ndaytemp: 15\nnighttemp: 9\ndaywind: 北\nnightwind: 北\ndaypower: 1-3\nnightpower: 1-3\ndaytemp_float: 15.0\nnighttemp_float: 9.0', metadata={'source': '/root/LangChainStudy/documentstore/t_china_city_weather.csv', 'row': 1}), Document(page_content='city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-02 13:33:28\ndate: 2024-05-02\ndayweather: 晴\nweek: 4\nnightweather: 晴\ndaytem

In [15]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
# all_splits = text_splitter.split_documents(data)

In [16]:
from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.vectorstores import Chroma
from chromadb.config import Settings
import chromadb
new_client = chromadb.EphemeralClient(settings=Settings(chroma_server_host="pt001",chroma_server_http_port="58000"))
vectorstore = Chroma.from_documents(client=new_client,documents=data, collection_name="test", embedding=QianfanEmbeddingsEndpoint())

In [17]:
# 基于prompt问题查找相似文档
print("prompt问题："+QUESTION1)
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION1)
[(document.page_content, score) for document, score in docs]

prompt问题：今天北京的天气怎么样,同时回答今天日期


[('city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 13:33:28\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweather: 晴\ndaytemp: 28\nnighttemp: 13\ndaywind: 西\nnightwind: 西\ndaypower: 1-3\nnightpower: 1-3\ndaytemp_float: 28.0\nnighttemp_float: 13.0',
  0.3622654141758377),
 ('city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 13:33:28\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweather: 晴\ndaytemp: 28\nnighttemp: 13\ndaywind: 西\nnightwind: 西\ndaypower: 1-3\nnightpower: 1-3\ndaytemp_float: 28.0\nnighttemp_float: 13.0',
  0.3622654141758377),
 ('city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 19:33:13\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweather: 晴\ndaytemp: 28\nnighttemp: 13\ndaywind: 西\nnightwind: 西\ndaypower: 1-3\nnightpower: 1-3\ndaytemp_float: 28.0\nnighttemp_float: 13.0',
  0.3605244121575497),
 ('city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 19:33:13\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweathe

In [18]:
from langchain.chains import RetrievalQA

from langchain_community.chat_models import QianfanChatEndpoint
from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate.from_template(CUSTOM_PROMPT_TEMPLATE)

llm = QianfanChatEndpoint(streaming=True,model="ERNIE-4.0-8K")
retriever=vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.0})
                                   
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
qa_chain({"query": QUESTION1})

{'query': '今天北京的天气怎么样,同时回答今天日期', 'result': '今天北京的天气是晴，同时今天是2024年5月7日。感谢你的提问。'}

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, return_source_documents=True)
result = qa_chain({"query": QUESTION1})
len(result['source_documents'])
result['source_documents']

[Document(page_content='city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 13:33:28\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweather: 晴\ndaytemp: 28\nnighttemp: 13\ndaywind: 西\nnightwind: 西\ndaypower: 1-3\nnightpower: 1-3\ndaytemp_float: 28.0\nnighttemp_float: 13.0', metadata={'row': 15, 'source': '/root/LangChainStudy/documentstore/t_china_city_weather.csv'}),
 Document(page_content='city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 13:33:28\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweather: 晴\ndaytemp: 28\nnighttemp: 13\ndaywind: 西\nnightwind: 西\ndaypower: 1-3\nnightpower: 1-3\ndaytemp_float: 28.0\nnighttemp_float: 13.0', metadata={'row': 15, 'source': '/root/LangChainStudy/documentstore/t_china_city_weather.csv'}),
 Document(page_content='city: 北京市\nadcode: 110000\nprovince: 北京\nreporttime: 2024-05-07 19:33:13\ndate: 2024-05-07\ndayweather: 晴\nweek: 2\nnightweather: 晴\ndaytemp: 28\nnighttemp: 13\ndaywind: 西\nnightwind: 西\ndaypower: 1-3\nnightp

In [20]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT})
qa(QUESTION1)

{'question': '今天北京的天气怎么样,同时回答今天日期',
 'chat_history': [SystemMessage(content='')],
 'answer': '今天北京的天气是晴天。今天是2024年5月7日。感谢你的提问。'}

In [21]:
qa(QUESTION2)

{'question': '今天北京市朝阳区的天气怎么样',
 'chat_history': [SystemMessage(content='人类询问今天北京的天气情况以及今天日期，AI回答今天北京是晴天，日期是2024年5月7日，并表示感谢提问。')],
 'answer': '今天北京市朝阳区的天气是白天多云，夜间阴。感谢你的提问。'}